#Material didático - Sensoriamento Remoto com Google Earth Engine - 3. Seleção de imagens
Objetivo: Recuperação de imagem, cobertura de nuvens, cobertura de neve.

Editor: Luizemara Szameitat (contato@lszameitat.com)

>Luizemara Szameitat é bolsista CAPES - Pós-Doutorado Estratégico 2022-2024.
O presente trabalho foi realizado com apoio da Coordenação de Aperfeiçoamento de Pessoal de Nível Superior - Brasil (CAPES) - Código de Financiamento 001.

# 1 - Preparar o ambiente e os dados

###1.1 - Bibliotecas e autorizações

In [46]:
!pip install geemap earthengine-api ipyleaflet -q

import os
import geemap
#import geemap.core as geemap
import ee
import matplotlib.pyplot as plt
import numpy as np


# GOOGLE COLAB - Adicionar chave do projeto do Google Engine
from google.colab import userdata
chave_do_projeto = userdata.get('EE_PROJECT_ID')
# Autenticação no Google Cloud
from google.colab import auth
auth.authenticate_user()
# Autenticação no Earth Engine
ee.Authenticate()
# Inicializer o Earth Engine com o ID
ee.Initialize(project = chave_do_projeto )

##1.2 - Escolha da imagem

###1.2.1 - Processamento

In [17]:
# AREA DE INTERESSE-----------------------------------------------------------#
# Google Maps https://maps.app.goo.gl/D9UskZDsAjvtGKiy6
# GOLD BAR MINING [-116.31248900966703, 39.77075791117339])
ponto = ee.Geometry.Point ([-116.46, 39.77075791117339])
raio = 1000  # m
area = ponto.buffer(raio).bounds()


Neste código escolhemos a imagem com base na menor cobertura de nuvens e menor cobertura neve.

In [18]:
# Filtra a coleção de imagens Landsat 5
collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterBounds(area) \
    .filterDate('1990-07-01', '1992-07-01') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

**Ajuste de escala**:

Esse processo ajusta os valores dos pixels das bandas da imagem Landsat para refletir valores reais de refletância e temperatura. Isto porque os valores brutos dos pixels precisam ser convertidos para unidades físicas que representem propriedades do solo e da vegetação.

A documentação oficial dos produtos Landsat tem os fatores de escala específicos para cada sensor e produto. No caso do Landsat 5, esses fatores são usados para converter os valores dos pixels para refletância e temperatura de brilho. Esses fatores são derivados de pesquisas e calibrações realizadas para garantir que os dados de satélite possam ser convertidos em valores físicos reais que representam condições do solo, vegetação e temperatura da superfície terrestre.

**Seleção das Bandas Ópticas e Multiplicação pelo Fator de Escala:**

*optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)*

* *image.select('SR_B.')*: Seleciona todas as bandas de refletância da imagem (SR_B1, SR_B2, ..., SR_B7).

* *.multiply(0.0000275)*: Multiplica os valores dos pixels por 0.0000275 para converter os valores dos pixels em unidades de refletância.

* *.add(-0.2)*: Adiciona -0.2 para ajustar a refletância.


***Seleção da Banda Térmica e Multiplicação pelo Fator de Escala:***

*thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)*

* *image.select('ST_B6')*: Seleciona a banda térmica ST_B6.

* *.multiply(0.00341802)*: Multiplica os valores dos pixels por 0.00341802 para converter os valores dos pixels em unidades de temperatura de brilho (Kelvin).

* *.add(149.0)*: Adiciona 149.0 para ajustar a temperatura de brilho.


Informações adicionais: Landsat Collection 2 Level-2 Science Products
https://pubs.usgs.gov/publication/fs20213055
Landsat Documentation https://www.usgs.gov/landsat-missions/landsat-project-documents



In [19]:
# Aplica os fatores de escala para a colecao Landsat 5
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(
        thermal_bands, None, True
    )
collection = collection.map(apply_scale_factors)

# Listar as bandas disponíveis
aux = collection.first()
band_info = aux.bandNames().getInfo()

print("Bandas disponíveis:")
for info in band_info:
    print(info)

Bandas disponíveis:
SR_B1
SR_B2
SR_B3
SR_B4
SR_B5
SR_B7
SR_ATMOS_OPACITY
SR_CLOUD_QA
ST_B6
ST_ATRAN
ST_CDIST
ST_DRAD
ST_EMIS
ST_EMSD
ST_QA
ST_TRAD
ST_URAD
QA_PIXEL
QA_RADSAT


Como é uma área de ocorrência de neve, adicionamos um índice (NDSI) para limitar imagens com muita neve. Este índice utiliza uma diferença entre as bandas 'B2' e 'B5' normalizadas.

In [20]:
# Calcula o índice de neve (NDSI) e adiciona para as imagens
def calculate_ndsi(image):
    ndsi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('NDSI')
    return image.addBands(ndsi)

collection = collection.map(calculate_ndsi)

# Define um limiar para considerar a cobertura de neve
def add_snow_coverage(image):
    ndsi = image.select('NDSI')
    snow_pixels = ndsi.gt(0.4)  # Treshold for snow detection
    snow_coverage = snow_pixels.reduceRegion(reducer=ee.Reducer.mean(), geometry=area, scale=30).get('NDSI')
    return image.set('snow_coverage', snow_coverage)

collection = collection.map(add_snow_coverage)

# Listar as bandas disponíveis
aux = collection.first()
band_info = aux.bandNames().getInfo()

print("Bandas disponíveis:")
for info in band_info:
    print(info)

Bandas disponíveis:
SR_B1
SR_B2
SR_B3
SR_B4
SR_B5
SR_B7
SR_ATMOS_OPACITY
SR_CLOUD_QA
ST_B6
ST_ATRAN
ST_CDIST
ST_DRAD
ST_EMIS
ST_EMSD
ST_QA
ST_TRAD
ST_URAD
QA_PIXEL
QA_RADSAT
NDSI


Agora podemos ordenar as imagens por menos neve e menos nuvens, e selecionar a imagem final

In [21]:
# Ordena as imagens pela menor cobertura de neve e pela menor cobertura de nuvens
sorted_images = collection.sort('__snow_coverage').sort('CLOUD_COVER')

# Seleciona a melhor imagem (a com menor cobertura de neve e nuvens)
melhor_imagem = sorted_images.first()

# Obtém informações da 'melhor imagem'
print('ID:', melhor_imagem.get('PRODUCT_ID').getInfo())
print('% de nuvens:', melhor_imagem.get('CLOUD_COVER').getInfo())
print('% de cobertura de neve:', melhor_imagem.get('snow_coverage').getInfo())

# Listar as bandas disponíveis
band_info = melhor_imagem.bandNames().getInfo()
print("Bandas disponíveis:")
for info in band_info:
    print(info)

ID: None
% de nuvens: 0
% de cobertura de neve: 0
Bandas disponíveis:
SR_B1
SR_B2
SR_B3
SR_B4
SR_B5
SR_B7
SR_ATMOS_OPACITY
SR_CLOUD_QA
ST_B6
ST_ATRAN
ST_CDIST
ST_DRAD
ST_EMIS
ST_EMSD
ST_QA
ST_TRAD
ST_URAD
QA_PIXEL
QA_RADSAT
NDSI


In [22]:
# Listar todas as propriedades disponíveis na imagem
propriedades = melhor_imagem.propertyNames().getInfo()
print("Propriedades disponíveis na imagem:")
for propriedade in propriedades:
    print(propriedade)


Propriedades disponíveis na imagem:
snow_coverage
IMAGE_QUALITY
WRS_TYPE
system:id
CORRECTION_GAIN_BAND_2
CORRECTION_GAIN_BAND_1
TEMPERATURE_ADD_BAND_ST_B6
system:footprint
REFLECTIVE_SAMPLES
TEMPERATURE_MINIMUM_BAND_ST_B6
CORRECTION_GAIN_BAND_7
CORRECTION_GAIN_BAND_6
CORRECTION_GAIN_BAND_5
SUN_AZIMUTH
CORRECTION_GAIN_BAND_4
CORRECTION_GAIN_BAND_3
DATE_ACQUIRED
ELLIPSOID
SENSOR_MODE
STATION_ID
DATA_SOURCE_REANALYSIS
ORIENTATION
WRS_ROW
CLOUD_COVER
GEOMETRIC_RMSE_VERIFY
COLLECTION_CATEGORY
GRID_CELL_SIZE_REFLECTIVE
ALGORITHM_SOURCE_SURFACE_TEMPERATURE
CLOUD_COVER_LAND
GEOMETRIC_RMSE_MODEL
COLLECTION_NUMBER
DATA_SOURCE_WATER_VAPOR
LANDSAT_SCENE_ID
WRS_PATH
L1_PROCESSING_SOFTWARE_VERSION
GEOMETRIC_RMSE_MODEL_Y
REFLECTIVE_LINES
GEOMETRIC_RMSE_MODEL_X
system:asset_size
DATA_SOURCE_AIR_TEMPERATURE
system:index
GEOMETRIC_RMSE_VERIFY_QUAD_LL
DATA_SOURCE_ELEVATION
SENSOR_ANOMALIES
GEOMETRIC_RMSE_VERIFY_QUAD_LR
REFLECTANCE_ADD_BAND_1
REFLECTANCE_ADD_BAND_2
DATUM
REFLECTANCE_ADD_BAND_3
REFLECTANC

###1.2.2 - Visualizar imagem escolhida

In [23]:
# Plotar
m = geemap.Map()
m.set_center(-116.45, 39.77075791117339, 8)
# Parâmetros de visualização
vis_params = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.0,
    'max': 0.3,
}
m.add_layer(melhor_imagem, vis_params, 'True Color (321)')
m

Map(center=[39.77075791117339, -116.45], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…

## Exportar o dado



In [64]:
# Aumenta o raio do buffer para garantir que a mina fique mais centralizada
# Localizar entorno da GOLD BAR MINING [-116.31248900966703, 39.77075791117339])
ponto = ee.Geometry.Point ([-116.31248900966703, 39.77075791117339])
raio = 9500  # m
area = ponto.buffer(raio).bounds()

# Recortar a imagem para o buffer definido
imagem_recortada = melhor_imagem.clip(area)

# Selecionar as bandas mais importantes
imagem_recortada = imagem_recortada.select(['SR_B1','SR_B2','SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])

m = geemap.Map()
m.set_center(-116.45, 39.77075791117339, 8)
# Parâmetros de visualização
vis_params = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.0,
    'max': 0.3,
}
m.add_layer(imagem_recortada, vis_params, 'Gold Bar Area - True Color (321)')
m


Map(center=[39.77075791117339, -116.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [66]:
# Exportar a imagem recortada para o Google Drive
tarefa = ee.batch.Export.image.toDrive(
    image=imagem_recortada,
    description='GoldBarMining',
    folder='SensoriamentoGEE',  # Substitua pelo nome da pasta no seu Google Drive
    fileNamePrefix='GoldBarMining',
    region=area,
    scale=30,
    crs='EPSG:4326',
    fileFormat='GeoTIFF'
)
tarefa.start()

# Monitorar o progresso da tarefa
import time
while tarefa.active():
    print('Aguardando exportação... ', tarefa.status())
    time.sleep(10)

print('Exportação concluída!')


Aguardando exportação...  {'state': 'READY', 'description': 'GoldBarMining', 'priority': 100, 'creation_timestamp_ms': 1721911451457, 'update_timestamp_ms': 1721911451457, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'ABCTT23WK4XUZUPRKBHFK3LJ', 'name': 'projects/sensoriamento1-430203/operations/ABCTT23WK4XUZUPRKBHFK3LJ'}
Aguardando exportação...  {'state': 'RUNNING', 'description': 'GoldBarMining', 'priority': 100, 'creation_timestamp_ms': 1721911451457, 'update_timestamp_ms': 1721911461479, 'start_timestamp_ms': 1721911458571, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'ABCTT23WK4XUZUPRKBHFK3LJ', 'name': 'projects/sensoriamento1-430203/operations/ABCTT23WK4XUZUPRKBHFK3LJ'}
Aguardando exportação...  {'state': 'RUNNING', 'description': 'GoldBarMining', 'priority': 100, 'creation_timestamp_ms': 1721911451457, 'update_timestamp_ms': 1721911461479, 'start_timestamp_ms': 1721911458571, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'ABCTT23WK4XUZUPRKBHFK3LJ', 'name': 